## 🔗 Merge maestro de OHLC horarios  
Este paso combina el `crypto_ohlc_join.csv` (dataset de referencia con métricas
fundamentales/narrativa, etc.) con los OHLC 📈 de **CryptoCompare (fuente
principal)**, Binance y CoinGecko:

* Se usa la clave `(symbol, timestamp)` donde `timestamp` son **segundos UTC**
  de cada hora.  
* Se hace un *outer merge* para **no perder ningún dato**; si una fuente no
  tiene esa hora, la celda queda `NaN`.
* Los nombres de columnas OHLC llevan prefijo:  
  `cc_open`, `bn_open`, `cg_open`, … para saber su procedencia.


In [7]:
import pandas as pd, numpy as np, pathlib, time, datetime as dt
from pandas.errors import EmptyDataError

# ────────────────────────── helpers ────────────────────────────────────────
def safe_read(fp: pathlib.Path, **kwargs) -> pd.DataFrame:
    """Lee CSV si existe y no está vacío; retorna DF vacío en otro caso."""
    if not fp.exists():
        print(f"⚠️  {fp.name} NO encontrado.")
        return pd.DataFrame()
    try:
        return pd.read_csv(fp, **kwargs)
    except EmptyDataError:
        print(f"⚠️  {fp.name} está vacío.  Se omite.")
        return pd.DataFrame()

def ensure_timestamp(df: pd.DataFrame) -> pd.DataFrame:
    """Asegura columna 'timestamp' (segundos UTC) a partir de 'date' o index."""
    if "timestamp" in df.columns:
        return df
    if "date" in df.columns:
        # aceptar varios formatos (p. ej. 2024-07-08 13:00:00 o solo fecha)
        df["timestamp"] = (
            pd.to_datetime(df["date"], errors="coerce", utc=True)
              .fillna(0).astype("int64") // 10**9
        )
    else:
        # como último recurso intentar usar el índice
        if df.index.dtype == "datetime64[ns]":
            df = df.reset_index().rename(columns={"index": "date"})
            df["timestamp"] = df["date"].astype("int64") // 10**9
    return df

def add_prefix(df: pd.DataFrame, pref: str) -> pd.DataFrame:
    """Renombra columnas OHLC con prefijo (open→pref_open, etc.)."""
    rename_map = {c: f"{pref}_{c}" for c in df.columns
                  if c in {"open","high","low","close","volume"}}
    return df.rename(columns=rename_map)

# ────────────────────────── rutas ──────────────────────────────────────────
base = pathlib.Path("")
paths = {
    "master":        base / "crypto_ohlc_join.csv",
    "cc":            base / "ohlc_hourly_cryptocompare.csv",
    "binance":       base / "ohlc_hourly_binance.csv",
    "gecko":         base / "ohlc_hourly_coingecko.csv",
}

# ────────────────────────── carga & preparación ────────────────────────────
master  = ensure_timestamp(safe_read(paths["master"]))
cc      = ensure_timestamp(safe_read(paths["cc"]))
binance = ensure_timestamp(safe_read(paths["binance"]))
gecko   = ensure_timestamp(safe_read(paths["gecko"]))

print("🗂  Filas leídas:",
      {k: len(v) for k,v in
       (("master", master),("cc",cc),("binance",binance),("gecko",gecko))})

# columnas mínimas para merge
merge_cols = ["symbol","timestamp"]

# ── mantener solo lo necesario de cada fuente y poner prefijos ─────────────
cc      = add_prefix(cc,  "cc")     [merge_cols + [c for c in cc.columns if c.startswith("cc_")]]
binance = add_prefix(binance,"bn")  [merge_cols + [c for c in binance.columns if c.startswith("bn_")]]
gecko   = add_prefix(gecko, "cg")   [merge_cols + [c for c in gecko.columns if c.startswith("cg_")]]

# ────────────────────────── merge incremental ──────────────────────────────
merged = master.copy()
for src_df in (cc, binance, gecko):
    if src_df.empty:          # si la fuente no tiene datos la saltamos
        continue
    merged = merged.merge(src_df, on=merge_cols, how="outer")

print("✅  Filas finales:", len(merged))

# ────────────────────────── guardar ────────────────────────────────────────
out_fp = base / "crypto_ohlc_join_hourly_full.csv"
merged.to_csv(out_fp, index=False)
print("💾 Guardado:", out_fp)


⚠️  ohlc_hourly_cryptocompare.csv NO encontrado.
⚠️  ohlc_hourly_binance.csv NO encontrado.
⚠️  ohlc_hourly_coingecko.csv NO encontrado.
🗂  Filas leídas: {'master': 55684, 'cc': 0, 'binance': 0, 'gecko': 0}


KeyError: "None of [Index(['symbol', 'timestamp'], dtype='object')] are in the [columns]"

## 1. Normalizar columnas básicas

In [4]:

def prep(df, src_name):
    if df.empty:
        return df
    # Unificar nombres
    rename_map = {
        'open':'open_'+src_name,
        'high':'high_'+src_name,
        'low':'low_'+src_name,
        'close':'close_'+src_name,
        'volume':'volume_'+src_name
    }
    df = df.rename(columns=rename_map)
    # Timestamp a UTC datetime sin zona
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True).dt.tz_convert(None)
    elif 'date' in df.columns:
        df['timestamp'] = pd.to_datetime(df['date'], utc=True).dt.tz_convert(None)
    # Símbolo upper
    if 'symbol' in df.columns:
        df['symbol'] = df['symbol'].str.upper()
    return df

master  = prep(master, 'master')
binance = prep(binance, 'binance')
cc      = prep(cc, 'cc')
gecko   = prep(gecko, 'gecko')


## 2. Combinar (outer join) sobre `symbol` + `timestamp`

In [5]:

from functools import reduce

dfs = [master, binance, cc, gecko]
dfs = [d for d in dfs if not d.empty]

full = reduce(lambda left,right: pd.merge(
                left, right, on=['symbol','timestamp'], how='outer', suffixes=('','')), dfs)

# Ordenar
full = full.sort_values(['symbol','timestamp']).reset_index(drop=True)
print("Full shape:", full.shape)


Full shape: (55684, 11)


## 3. Guardar

In [6]:

out_fp = DATA_DIR / "crypto_ohlc_join_full.csv"
full.to_csv(out_fp, index=False)
print("📑 Guardado", out_fp.relative_to(DATA_DIR.parent))


📑 Guardado crypto_ohlc_join_full.csv
